# Merge Datasource

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import io , os , time ,requests,json,bs4
from time import sleep
import lxml.etree as xml
import pandas_datareader as pdr
from yahoo_finance import Share
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# init 
datapath = '/mnt/c/Users/poom/drive/01project/01cap_stone/04data/01stockprice'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [7]:
## import data
# Historical stockprice and technical indicator
sp500histprice = pd.read_csv(os.path.join(datapath,'histprice3.csv'))
# sentimental analysis
sentimental = pd.read_csv(os.path.join(datapath,'sentimental_analysis.csv'))
# GG trends
ggtrend = pd.read_csv(os.path.join(datapath,'GGTrendallTimeMonthly.csv'))
# World indicies
indices = pd.read_csv(os.path.join(datapath,'General_data.csv'))